In [1]:
import gurobipy as gp

In [2]:
# Classe auxiliar para controlar cada paciente

class Patient:
    def __init__(self, chance : int, pUTI : int, pUTSI : int, pUTP : int) -> None:
        self.survivingChance : int = chance
        self.priorities : dict = { "UTI" : pUTI, "UTSI" : pUTSI, "UTP" : pUTP }
        self.allocated : bool = False

In [3]:
# Classe auxiliar para o controle de tipos e controle de camas
# A refatorar

class Unities:
    def __init__(self, uti : int, utsi : int, utp : int) -> None:
        self.types : list = [ "UTI", "UTSI", "UTP" ] #Útil
        self.qtdTypes : dict = { "UTI" : uti, "UTSI" : utsi, "UTP" : utp } #Útil

        self.setUTI : list = [ f"Bed_UTI_{i}" for i in range( self.qtdTypes["UTI"] ) ]
        self.setUTSI : list = [ f"Bed_UTSI_{i}" for i in range( self.qtdTypes["UTSI"] ) ]
        self.setUTP : list = [ f"Bed_UTP_{i}" for i in range( self.qtdTypes["UTP"] ) ]
        
        self.setBeds_ : list = [ self.setUTI, self.setUTSI, self.setUTP ]
        self.listBeds : list = self.setBeds()
        self.dictBeds : dict = { "UTI" : self.setUTI, "UTSI" : self.setUTSI, "UTP" : self.setUTP }
        self.dBeds = self.getDictBed() # Útil

    def setBeds(self) -> list:
        set = list()
        for i in range( self.qtdTypes["UTI"] ):
            set.append("UTI_")
        for i in range( self.qtdTypes["UTSI"] ):
            set.append("UTSI_")
        for i in range( self.qtdTypes["UTP"] ):
            set.append("UTP_")
        return set
    
    def getDictBed(self) -> dict:
        d = dict()

        for i in self.setUTI:
            d[i] = i
        for i in self.setUTSI:
            d[i] = i
        for i in self.setUTP:
            d[i] = i
        
        return d

In [4]:
# Lê os dados do arquivo de instâncias

def readData(file_name: str):
    file = open(file_name, "r")

    rows = file.readlines()
    values = rows[0].strip().split()
    qtd_Patients = int(values[0])

    del(rows[0])

    vet_Patients = list()
    unities = rows[0].strip().split()
    for i in range(len(unities)):
        unities[i] = int(unities[i])

    del(rows[0])

    patient : Patient

    dictPatients = dict()
    lsPatients = list()
    for row in rows:
        [Pi, Pi0, Pi1, Pi2] = row.strip().split()
        patient = Patient(int(Pi), int(Pi0), int(Pi1), int(Pi2))
        lsPatients.append( patient )

    for i in range(len(lsPatients)):
        dictPatients.update( { f"Patient_{i}" : lsPatients[i] } )
    
    uti = unities[0]
    utsi = unities[1]
    utp = unities[2]
    
    return lsPatients, uti, utsi, utp, qtd_Patients

In [9]:
# Solver propriamente dito

def solverHU(file_path: str) -> None:

    lsPatients, uti, utsi, utp, qtd_Patients = readData(file_path)

    qtdBeds = uti + utsi + utp

    patients = list(lsPatients)
    unities = Unities( uti, utsi, utp )

    m = gp.Model()

    x = m.addVars(patients, unities.types, unities.dBeds, vtype = gp.GRB.BINARY)

    # Obective function
    m.setObjective(
        gp.quicksum(
            gp.quicksum(
                gp.quicksum(
                    x[i, j, k] * i.survivingChance for k in unities.dictBeds[j]
                ) for j in unities.types
            ) for i in patients
        ),
        sense = gp.GRB.MAXIMIZE
    )

    #Constraints
    c1 = list()
    for j in unities.types:
        for k in unities.dictBeds[j]:
            c1.append(
                m.addConstr(
                    gp.quicksum( x[i, j, k] for i in patients ) <= 1
                )
            )
    # Garante que só haja um paciente alocado em cada leito

    c2 = list()
    for i in patients:
        c2.append(
            m.addConstr(
                gp.quicksum(
                    gp.quicksum( x[i, j, k] for k in unities.dictBeds[j]) \
                        for j in unities.types
                ) <= 1
            )
        )
    # Garante que o mesmo paciente não seja alocado em leitos diferentes

    c3 = list()
    for j in unities.types:
        c3.append(
            m.addConstrs(
            gp.quicksum( x[i, j, k] for k in unities.dictBeds[j] ) <= unities.qtdTypes[j] \
                for i in patients
            )
        )
    # Garante que não sejam alocadas mais pacientes do que há de camas disponíveis para cada tipo

    c4 =list()
    for i1 in patients:
        for i2 in patients:
            for j in unities.types:
                for k in unities.dictBeds[j]:
                    if i1.priorities[j] > i2.priorities[j]:
                        c4.append(m.addConstr(
                            x[i1, j, k] >= x[i2, j, k]
                            )
                        )
    # Garante que um paciente com maior prioridade em relação a outro deve ser alocado em um leito antes do de 
    # menor prioridade

    #Execute
    m.optimize()

    # Apenas operações para imprimir na tela os resultados
    
    countVars = 0
    for bedType in unities.types:
        for bed in unities.dictBeds[bedType]:
            for patient in patients:
                print( x[patient, bedType, bed].X, bedType, bed, patient, sep="\t")
                countVars += 1
    
    qtdAlocated = 0
    totalChance = 0

    qtdUTI = 0
    qtdUTSI = 0
    qtdUTP = 0

    for bedType in unities.types:
        for bed in unities.dictBeds[bedType]:
            for patient in patients:
                if x[patient, bedType, bed].X == 1:
                    totalChance += patient.survivingChance
                    qtdAlocated += 1
                    patient.allocated = True
                    if bedType == "UTI":
                        qtdUTI += 1
                    elif bedType == "UTSI":
                        qtdUTSI += 1
                    elif bedType == "UTP":
                        qtdUTP += 1
                    print(patient.survivingChance)
                    print(patient.priorities["UTI"], patient.priorities["UTSI"], patient.priorities["UTP"])
                    print(bedType, bed)

    print(f"Quantidade alocada: {qtdAlocated}, Somatório total objetivo: {totalChance}, Total de variáveis: {countVars}")

In [10]:
solverHU("Inst_Patients/Inst_0.txt")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i3-5005U CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 204 rows, 162 columns and 486 nonzeros
Model fingerprint: 0x85a1cf2b
Variable types: 0 continuous, 162 integer (162 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 184.0000000
Presolve removed 198 rows and 154 columns
Presolve time: 0.01s
Presolved: 6 rows, 8 columns, 16 nonzeros
Variable types: 0 continuous, 8 integer (8 binary)

Root relaxation: cutoff, 3 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0       184.00000 